In [52]:
%%sh
# build the image
cd container/

# tag it as example-image:latest
docker build -t example-serve:latest .


Step 1/11 : FROM ubuntu:18.04
 ---> c090eaba6b94
Step 2/11 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> dd1348b3a447
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5d88b1be1abc
Step 4/11 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 6417ac44ded6
Step 5/11 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 09ada8830cc5
Step 6/11 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 2885b6a01bd3
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> ab2910812f0d
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 0c0e511ac354
Step 9/11 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 10fcf3254c9d
Step 10/11 : COPY src /opt/progra

In [45]:
from subprocess import Popen
import subprocess

cmd = "docker run -p 8080:8080 --rm example-serve:latest serve"


process = subprocess.Popen(cmd, shell=True,
                           stdout=subprocess.PIPE, 
                           stderr=subprocess.PIPE,
                           start_new_session=True
                          )


process.wait()

# wait for the process to terminate
out, err = process.communicate()
errcode = process.returncode

print(out.decode('utf-8'))
print(process.pid)

KeyboardInterrupt: 

In [44]:
import os

os.system('ls .')

0

In [38]:
import asyncio

proc = await asyncio.create_subprocess_exec(
    'ls','.',
    stdout=asyncio.subprocess.PIPE,
    stderr=asyncio.subprocess.PIPE)


# if proc takes very long to complete, the CPUs are free to use cycles for 
# other processes
#stdout, stderr = await proc.communicate()
stdout, stderr = proc.communicate()
proc.returncode
# 0

# must call decode because stdout is a bytes object
stdout.decode()
# total 24K
# drwxrwxr-x  3 felipe felipe 4,0K Nov  4 17:52 .
# drwxrwxr-x 39 felipe felipe 4,0K Nov  3 18:31 ..
# drwxrwxr-x  2 felipe felipe 4,0K Nov  3 19:32 .ipynb_checkpoints
# -rw-rw-r--  1 felipe felipe  11K Nov  4 17:52 main.ipynb

stderr.decode()
# ''  empty string   

AssertionError: yield from wasn't used with future

TypeError: decode() argument 1 must be str, not int

In [46]:
#help(Popen)

In [62]:
%%bash
# run the inference container
docker run -p 8080:8080 --rm example-serve:latest serve 

Process is terminated.


In [57]:
%%bash
curl -H "Content-Type: text/csv" -v http://localhost:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8080...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8080 (#0)
> GET /ping HTTP/1.1
> Host: localhost:8080
> User-Agent: curl/7.68.0
> Accept: */*
> Content-Type: text/csv
> 
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.14.0 (Ubuntu)
< Date: Wed, 10 Mar 2021 02:11:47 GMT
< Content-Type: application/json
< Content-Length: 1
< Connection: keep-alive
< 
{ [1 bytes data]
100     1  100     1    0     0   1000      0 --:--:-- --:--:-- --:--:--  1000
* Connection #0 to host localhost left intact


In [61]:
%%bash
curl --data-binary @container/local_test/payload.csv -H "Content-Type: text/csv" -v http://localhost:8080/invocations

I just take json, and I am fed with text/csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 127.0.0.1:8080...
* TCP_NODELAY set
* Connected to localhost (127.0.0.1) port 8080 (#0)
> POST /invocations HTTP/1.1
> Host: localhost:8080
> User-Agent: curl/7.68.0
> Accept: */*
> Content-Type: text/csv
> Content-Length: 8
> 
} [8 bytes data]
* upload completely sent off: 8 out of 8 bytes
* Mark bundle as not supporting multiuse
< HTTP/1.1 200 OK
< Server: nginx/1.14.0 (Ubuntu)
< Date: Wed, 10 Mar 2021 02:15:02 GMT
< Content-Type: text/plain; charset=utf-8
< Content-Length: 44
< Connection: keep-alive
< 
{ [44 bytes data]
100    52  100    44  100     8  14666   2666 --:--:-- --:--:-- --:--:-- 17333
* Connection #0 to host localhost left intact


In [19]:
import boto3
import datetime
import pprint
import os
import time

pp = pprint.PrettyPrinter(indent=1)
iam = boto3.client('iam')
ecr = boto3.client('ecr')

image_name="example-inference"

try:
    # The repository might already exist
    # in your ECR
    cr_res = ecr.create_repository(
        repositoryName=image_name)
    pp.pprint(cr_res)
except Exception as e:
    print(e)

An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'example-inference' already exists in the registry with id '688520471316'


In [20]:
%%bash
account=$(aws sts get-caller-identity --query Account | sed -e 's/^"//' -e 's/"$//')
region=$(aws configure get region)
ecr_account=${account}.dkr.ecr.${region}.amazonaws.com

# Give docker your ECR login password
aws ecr get-login-password --region $region | docker login --username AWS --password-stdin $ecr_account

# Fullname of the repo
fullname=$ecr_account/example-inference:latest

#echo $fullname
# Tag the image with the fullname
docker tag example-inference:latest $fullname

# Push to ECR
docker push $fullname

Login Succeeded
The push refers to repository [688520471316.dkr.ecr.us-west-2.amazonaws.com/example-inference]
f4cd793a4ebc: Preparing
b520e3bd5eba: Preparing
3a3b4090fe28: Preparing
5b850ff3c508: Preparing
408c63ea099b: Preparing
9f10818f1f96: Preparing
27502392e386: Preparing
c95d2191d777: Preparing
27502392e386: Waiting
c95d2191d777: Waiting
3a3b4090fe28: Pushed
f4cd793a4ebc: Pushed
5b850ff3c508: Pushed
9f10818f1f96: Pushed
27502392e386: Pushed
c95d2191d777: Pushed
408c63ea099b: Pushed
b520e3bd5eba: Pushed
latest: digest: sha256:72907655e9cc8dd5a940f7c04932e66aee7890356d6df4bad7380979a362ae2c size: 1989


WARNING! Your password will be stored unencrypted in /home/ubuntu/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



In [22]:
# Inspect the ECR repository
repo_res = ecr.describe_images(
    repositoryName='example-inference')
pp.pprint(repo_res)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '404',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 00:42:52 GMT',
                                      'x-amzn-requestid': '3073c552-5b39-4217-9895-6dee8f9260e3'},
                      'HTTPStatusCode': 200,
                      'RequestId': '3073c552-5b39-4217-9895-6dee8f9260e3',
                      'RetryAttempts': 0},
 'imageDetails': [{'artifactMediaType': 'application/vnd.docker.container.image.v1+json',
                   'imageDigest': 'sha256:72907655e9cc8dd5a940f7c04932e66aee7890356d6df4bad7380979a362ae2c',
                   'imageManifestMediaType': 'application/vnd.docker.distribution.manifest.v2+json',
                   'imagePushedAt': datetime.datetime(2021, 3, 5, 0, 31, 31, tzinfo=tzlocal()),
                   'imageSizeInBytes': 132463690,
                   'imageTags': ['latest'],
                   're

In [24]:
image_uri="688520471316.dkr.ecr.us-west-2.amazonaws.com/example-inference:latest"
role_arn = 'arn:aws:iam::688520471316:role/sm'

In [25]:
sm_boto3 = boto3.client('sagemaker')

cm_res = sm_boto3.create_model(
    ModelName='example-inference',
    Containers=[
        {
            'Image': image_uri,
   
        },
    ],
    ExecutionRoleArn=role_arn,
    EnableNetworkIsolation=False
)

pp.pprint(cm_res)


{'ModelArn': 'arn:aws:sagemaker:us-west-2:688520471316:model/example-inference',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '79',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 00:46:16 GMT',
                                      'x-amzn-requestid': '88ceba30-6a67-4c52-b2d9-71fef75121af'},
                      'HTTPStatusCode': 200,
                      'RequestId': '88ceba30-6a67-4c52-b2d9-71fef75121af',
                      'RetryAttempts': 0}}


In [ ]:
# create endpoint config
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint
model_name='example-inference'
initial_instance_count=1
instance_type='ml.t2.medium'
variant_name = "AMeaningfulProdVarName" #^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}

# why do we need it
# think about a use case, where we need many variants
production_variants = [
    {
        "VariantName": variant_name,
        "ModelName": model_name,
        "InitialInstanceCount": initial_instance_count,
        "InstanceType": instance_type
    }
]

endpoint_config_name = "ExampleInferenceConfig" #^[a-zA-Z0-9](-*[a-zA-Z0-9]){0,62}

endpoint_config = {
    "EndpointConfigName": endpoint_config_name,
    "ProductionVariants": production_variants,
}

ep_conf_res = sm_boto3.create_endpoint_config(**endpoint_config)


In [31]:
pp.pprint(ep_conf_res)

{'EndpointConfigArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint-config/exampleinferenceconfig',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '103',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:44:02 GMT',
                                      'x-amzn-requestid': 'fd66804f-380a-478e-b3fb-b3f6296f0639'},
                      'HTTPStatusCode': 200,
                      'RequestId': 'fd66804f-380a-478e-b3fb-b3f6296f0639',
                      'RetryAttempts': 0}}


In [32]:
# create endpoint
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint


endpoint_name='exmaple-endpoint'
ep_res = sm_boto3.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name
    )

In [33]:
pp.pprint(ep_res)

{'EndpointArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint/exmaple-endpoint',
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '84',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:53:40 GMT',
                                      'x-amzn-requestid': '641e22b1-dc16-4d50-b615-86a857180def'},
                      'HTTPStatusCode': 200,
                      'RequestId': '641e22b1-dc16-4d50-b615-86a857180def',
                      'RetryAttempts': 0}}


In [35]:
# describe endpoint
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.describe_endpoint

ep_des_res = sm_boto3.describe_endpoint(
    EndpointName=endpoint_name
)

pp.pprint(ep_des_res)

{'CreationTime': datetime.datetime(2021, 3, 5, 1, 53, 41, 572000, tzinfo=tzlocal()),
 'EndpointArn': 'arn:aws:sagemaker:us-west-2:688520471316:endpoint/exmaple-endpoint',
 'EndpointConfigName': 'ExampleInferenceConfig',
 'EndpointName': 'exmaple-endpoint',
 'EndpointStatus': 'Creating',
 'LastModifiedTime': datetime.datetime(2021, 3, 5, 1, 53, 41, 572000, tzinfo=tzlocal()),
 'ResponseMetadata': {'HTTPHeaders': {'content-length': '260',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 01:56:13 GMT',
                                      'x-amzn-requestid': '285fb80a-96dc-4adc-aceb-385d729a0199'},
                      'HTTPStatusCode': 200,
                      'RequestId': '285fb80a-96dc-4adc-aceb-385d729a0199',
                      'RetryAttempts': 0}}


Invoke endpoint 

```sh
grep -rwn .  -e predict
```

```python
def predict(
        self, data, initial_args=None, target_model=None, target_variant=None, inference_id=None
    ):
        """Return the inference from the specified endpoint.
        Args:
            data (object): Input data for which you want the model to provide
                inference. If a serializer was specified when creating the
                Predictor, the result of the serializer is sent as input
                data. Otherwise the data must be sequence of bytes, and the
                predict method then sends the bytes in the request body as is.
            initial_args (dict[str,str]): Optional. Default arguments for boto3
                ``invoke_endpoint`` call. Default is None (no default
                arguments).
            target_model (str): S3 model artifact path to run an inference request on,
                in case of a multi model endpoint. Does not apply to endpoints hosting
                single model (Default: None)
            target_variant (str): The name of the production variant to run an inference
                request on (Default: None). Note that the ProductionVariant identifies the
                model you want to host and the resources you want to deploy for hosting it.
            inference_id (str): If you provide a value, it is added to the captured data
                when you enable data capture on the endpoint (Default: None).
        Returns:
            object: Inference for the given input. If a deserializer was specified when creating
                the Predictor, the result of the deserializer is
                returned. Otherwise the response returns the sequence of bytes
                as is.
        """

        request_args = self._create_request_args(
            data, initial_args, target_model, target_variant, inference_id
        )
        response = self.sagemaker_session.sagemaker_runtime_client.invoke_endpoint(**request_args)
        return self._handle_response(response)
```

so `sagemaker_session.sagemaker_runtime_client` invokes the endpint

I searched `sagemaker_runtime` in session.py

```python
def _initialize(
        self,
        boto_session,
        sagemaker_client,
        sagemaker_runtime_client,
        sagemaker_featurestore_runtime_client,
    ):
        """Initialize this SageMaker Session.
        Creates or uses a boto_session, sagemaker_client and sagemaker_runtime_client.
        Sets the region_name.
        """
        self.boto_session = boto_session or boto3.DEFAULT_SESSION or boto3.Session()

        self._region_name = self.boto_session.region_name
        if self._region_name is None:
            raise ValueError(
                "Must setup local AWS configuration with a region supported by SageMaker."
            )

        self.sagemaker_client = sagemaker_client or self.boto_session.client("sagemaker")
        prepend_user_agent(self.sagemaker_client)

        if sagemaker_runtime_client is not None:
            self.sagemaker_runtime_client = sagemaker_runtime_client
        else:
            config = botocore.config.Config(read_timeout=80)
            self.sagemaker_runtime_client = self.boto_session.client(
                "runtime.sagemaker", config=config
            )

        prepend_user_agent(self.sagemaker_runtime_client)

        if sagemaker_featurestore_runtime_client:
            self.sagemaker_featurestore_runtime_client = sagemaker_featurestore_runtime_client
        else:
            self.sagemaker_featurestore_runtime_client = self.boto_session.client(
                "sagemaker-featurestore-runtime"
            )

        self.local_mode = False
 
```

This led to investigate [`SageMakerRuntime` client in boto3](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker-runtime.html)


```
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(
    EndpointName='string',
    Body=b'bytes'|file,
    ContentType='string',
    Accept='string',
    CustomAttributes='string',
    TargetModel='string',
    TargetVariant='string',
    TargetContainerHostname='string',
    InferenceId='string'
)
```

To figure out how content type looks like, I looked up [`serializers.py`](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/serializers.py) and [`deserializers.py`](https://github.com/aws/sagemaker-python-sdk/blob/master/src/sagemaker/deserializers.py)


Why is it called SageMaker runtime

In [ ]:
# how to serialize csv

import csv
import io

def _serialize_row(data):
    """Serialize data as a CSV-formatted row.
    Args:
        data (sting): Data to be serialized in a row.
    Returns:
        str: The data serialized as a CSV-formatted row.
    """
    if isinstance(data, str):
        return data

    if isinstance(data, np.ndarray):
        data = np.ndarray.flatten(data)

    if hasattr(data, "__len__"):
        if len(data) == 0:
            raise ValueError("Cannot serialize empty array")
        csv_buffer = io.StringIO()
        csv_writer = csv.writer(csv_buffer, delimiter=",")
        csv_writer.writerow(data)
        return csv_buffer.getvalue().rstrip("\r\n")
        
csv_buffer = io.StringIO()
csv_writer = csv.writer(csv_buffer, delimiter=',')
csv_writer.writerow('xyze')
v = csv_buffer.getvalue().rstrip('\r\n')

print(v.rstrip("\r\n"))

In [63]:
# invoke endpoint
import json

sm_runtime = boto3.client('sagemaker-runtime')

body=json.dumps('a json string')

# the model only supports csv data (look at how the model is defined in container/predictor.py)
content_type='text/csv'
# see the cell below for serializing a string into csv format

csv_buffer = io.StringIO()
csv_writer = csv.writer(csv_buffer, delimiter=',')
csv_writer.writerow('xyze')
body = csv_buffer.getvalue().rstrip('\r\n')

# respnse type is also text/csv (look at decision_tree/predictor.py)
accept='text/csv'

res=sm_runtime.invoke_endpoint(
    EndpointName=endpoint_name,
    Body=body,                # encoded input data
    ContentType=content_type, # I told the endpoint what's the encode
    Accept=accept             # I told the endpoint how I wish to decode its response
)

print(res)

{'ResponseMetadata': {'RequestId': '94053c6a-3290-4562-9089-d78dd65e9a77', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '94053c6a-3290-4562-9089-d78dd65e9a77', 'x-amzn-invoked-production-variant': 'AMeaningfulProdVarName', 'date': 'Fri, 05 Mar 2021 03:10:19 GMT', 'content-type': 'text/csv; charset=utf-8', 'content-length': '26'}, 'RetryAttempts': 0}, 'ContentType': 'text/csv; charset=utf-8', 'InvokedProductionVariant': 'AMeaningfulProdVarName', 'Body': <botocore.response.StreamingBody object at 0x7f46e9c84d68>}


In [56]:
# how to decode json string

import codecs

class SimpleBaseDeserializer:
    pass

class JSONDeserializer(SimpleBaseDeserializer):
    """Deserialize JSON data from an inference endpoint into a Python object."""

    def __init__(self, accept="application/json"):
        """Initialize a ``JSONDeserializer`` instance.
        Args:
            accept (union[str, tuple[str]]): The MIME type (or tuple of allowable MIME types) that
                is expected from the inference endpoint (default: "application/json").
        """
        super(JSONDeserializer, self).__init__(accept=accept)

    def deserialize(self, stream, content_type):
        """Deserialize JSON data from an inference endpoint into a Python object.
        Args:
            stream (botocore.response.StreamingBody): Data to be deserialized.
            content_type (str): The MIME type of the data.
        Returns:
            object: The JSON-formatted data deserialized into a Python object.
        """
        try:
            return json.load(codecs.getreader("utf-8")(stream))
        finally:
            stream.close()
            
            

class CSVDeserializer(SimpleBaseDeserializer):
    """Deserialize a stream of bytes into a list of lists.
    Consider using :class:~`sagemaker.deserializers.NumpyDeserializer` or
    :class:~`sagemaker.deserializers.PandasDeserializer` instead, if you'd like to convert text/csv
    responses directly into other data types.
    """

    def __init__(self, encoding="utf-8", accept="text/csv"):
        """Initialize a ``CSVDeserializer`` instance.
        Args:
            encoding (str): The string encoding to use (default: "utf-8").
            accept (union[str, tuple[str]]): The MIME type (or tuple of allowable MIME types) that
                is expected from the inference endpoint (default: "text/csv").
        """
        super(CSVDeserializer, self).__init__(accept=accept)
        self.encoding = encoding

    def deserialize(self, stream, content_type):
        """Deserialize data from an inference endpoint into a list of lists.
        Args:
            stream (botocore.response.StreamingBody): Data to be deserialized.
            content_type (str): The MIME type of the data.
        Returns:
            list: The data deserialized into a list of lists representing the
                contents of a CSV file.
        """
        try:
            decoded_string = stream.read().decode(self.encoding)
            return list(csv.reader(decoded_string.splitlines()))
        finally:
            stream.close()
        

In [64]:
# decode response
res_body = res['Body']
res_body.read().decode('utf-8')

'You are a wonderful potato'

In [65]:
# SageMaker Pricing

In [66]:
del_res = sm_boto3.delete_endpoint(
    EndpointName=endpoint_name)
pp.pprint(del_res)

{'ResponseMetadata': {'HTTPHeaders': {'content-length': '0',
                                      'content-type': 'application/x-amz-json-1.1',
                                      'date': 'Fri, 05 Mar 2021 03:16:19 GMT',
                                      'x-amzn-requestid': '42acf2c7-6c9e-4228-99cf-21e7bf684c4e'},
                      'HTTPStatusCode': 200,
                      'RequestId': '42acf2c7-6c9e-4228-99cf-21e7bf684c4e',
                      'RetryAttempts': 0}}


# Create Endpoint 

In this notebook, you will learn basics about hosting your trained model on Amazon SageMaker for inference. There are two ways you can use Amazon SageMaker for inference:
1. Set up persistent endpoint for real-time online inference
2. Gather data to be predicted in batch and use SageMaker batch transform for offline inference. 

In this notebook, we focus on the first option and we will discuss batch transform in another notebook. 

You are highly recommeneded to go through [the section on model deployment](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-deployment.html) in the official docs before moving on.


The pricing for setting up an endpoint can be found [here](https://aws.amazon.com/sagemaker/pricing/)

Like a [CreateTrainingJob](https://github.com/hsl89/amazon-sagemaker-examples/blob/sagemaker-fundamentals/sagemaker-fundamentals/create-training-job/create-training-job.ipynb), Amazon SageMaker interacts with your inference logic via a containerized enviornment. 

The following APIs are relavent:
* [`CreateModel`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_model)
* [`CreateEndpointConfig`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint_config)
* [`CreateEndpoint`](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/sagemaker.html#SageMaker.Client.create_endpoint)

You are highly recommended to go through them. It's okay if you don't understand everything, we will go through them in detail in this notebook. 

The outline of this notebook is:
1. Build an inference image
2. Test the inference image / container locally and push it to ECR
3. Use the ECR address of the inference container to define a model by calling `CreateModel`
4. Specify configuration of an endpoint by calling `CreateEndpointConfig`
5. Use model definition from 3 and endpoint configuration from 4 to create an endpoint by calling `CreateEndpoint`
6. Invoke the endpoint by using SageMaker runtime client 

## Build an inference image

You inference image must be a self-contained web server. When you run your inference container locally, it should listen on port 8080 and accept POST requests to the `/invocations` endpoint. The payload of the POST requests is the content of the data that you want your model to predict. Since the inference container is essentially a web server, you should expect it to look differently from the container we used for [`CreateTrainingJob`](https://github.com/hsl89/amazon-sagemaker-examples/blob/sagemaker-fundamentals/sagemaker-fundamentals/create-training-job/create-training-job.ipynb). 

In this notebook, we use a minimal python stack to build our web server:
![Request serving stack](stack.png)

### Further readings on the serving stack

* [Overview of the stack](https://flask.palletsprojects.com/en/1.1.x/deploying/uwsgi/)
* [Ngnix homepage](https://www.nginx.com/resources/wiki/start/) 
* [WSGI homepage](https://gunicorn.org/)
* [Flask homepage](https://flask.palletsprojects.com/en/1.1.x/)

### How SageMaker runs your container

SageMaker runs your container like

```sh
docker run <image> serve
```

This means you need to have an executable called `serve` in the `PATH`. In this notebook, we will create a python script as an **executable** and put it in the working directory of the docker image. 
        
The folder `container/src` contains the configs and entry point of the web server

In [1]:
!ls  container/src

nginx.conf  predictor.py  serve  wsgi.py


#### Entrypoint for Ngnixs server

`serve` is a python executable that is intended to be used as the entrypoint for the inference image.

In [5]:
!cat container/src/serve

#!/usr/bin/env python

# This file implements the scoring service shell. You don't necessarily need to modify it for various
# algorithms. It starts nginx and gunicorn with the correct configurations and then simply waits until
# gunicorn exits.
#
# The flask server is specified to be the app object in wsgi.py
#
# We set the following parameters:
#
# Parameter                Environment Variable              Default Value
# ---------                --------------------              -------------
# number of workers        MODEL_SERVER_WORKERS              the number of CPU cores
# timeout                  MODEL_SERVER_TIMEOUT              60 seconds

import multiprocessing
import os
import signal
import subprocess
import sys

cpu_count = multiprocessing.cpu_count()

model_server_timeout = os.environ.get('MODEL_SERVER_TIMEOUT', 60)
model_server_workers = int(os.environ.get('MODEL_SERVER_WORKERS', cpu_count))

def sigterm_handler(nginx_pid, gunicorn_pid):
    try:
        os.kill(nginx_p

#### Config file for Ngnix server
`nginx.conf` is the config file for the nginx server.

In [1]:
!cat container/src/nginx.conf

worker_processes 1;
daemon off; # Prevent forking


pid /tmp/nginx.pid;
error_log /var/log/nginx/error.log;

events {
  # defaults
}

http {
  include /etc/nginx/mime.types;
  default_type application/octet-stream;
  access_log /var/log/nginx/access.log combined;
  
  upstream gunicorn {
    server unix:/tmp/gunicorn.sock;
  }

  server {
    listen 8080 deferred;
    client_max_body_size 5m;

    keepalive_timeout 5;
    proxy_read_timeout 1200s;

    location ~ ^/(ping|invocations) {
      proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
      proxy_set_header Host $http_host;
      proxy_redirect off;
      proxy_pass http://gunicorn;
    }

    location / {
      return 404 "{}";
    }
  }
}


#### WSGI config

In [10]:
!cat container/src/wsgi.py

import predictor as myapp

# This is just a simple wrapper for gunicorn to find your app.
# If you want to change the algorithm file, simply change "predictor" above to the
# new file.

app = myapp.app


#### Inference logic

The most important file in `container/src` is `predictor.py`. It contains the inference logic. Other files in the `container/src` can be used **as it**. But you will need to customize `predictor.py` to implement your own inference logic. 

In [11]:
!pygmentize container/src/predictor.py

# This is the file that implements a flask server to do inferences. It's the file that you will modify to adapt to your own inference logic.

import flask

# The flask app for serving predictions
app = flask.Flask(__name__)

@app.route('/ping', methods=['GET'])
def ping():
    """Determine if the container is working and healthy. 
    In this sample container, we declare
    it healthy if we can load the model successfully."""

    status = 200
    return flask.Response(response='\n', 
        status=status, mimetype='application/json')

@app.route('/invocations', methods=['POST'])
def inference():
    """Do an inference on incoming data. 
    In this sample server, we take data as application/json,
    print it out to confirm that the server received it.  
    """
    content_type = flask.request.content_type
    if flask.request.content_type != "application/json":
        msg = "I just take json, and I am fed with {}".format(
            content_type)
    else:
        msg = "I am fe

## Build the container

We build the container from `container/Dockderfile`. And let's call this image `example-serve`. 

In [2]:
!cat container/Dockerfile

# Build an image that can do training and inference in SageMaker
# This is a Python 3 image that uses the nginx, gunicorn, flask stack
# for serving inferences in a stable way.

FROM ubuntu:18.04

MAINTAINER Amazon AI <sage-learner@amazon.com>


RUN apt-get -y update && apt-get install -y --no-install-recommends \
         wget \
         python3-pip \
         python3-setuptools \
         nginx \
         ca-certificates \
    && rm -rf /var/lib/apt/lists/*

RUN ln -s /usr/bin/python3 /usr/bin/python
RUN ln -s /usr/bin/pip3 /usr/bin/pip

# Here we get all python packages.
# There's substantial overlap between scipy and numpy that we eliminate by
# linking them together. Likewise, pip leaves the install caches populated which uses
# a significant amount of space. These optimizations save a fair amount of space in the
# image, which reduces start up time.
RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn

# Set some environment variabl

In [3]:
%%sh
# build the image
cd container/

# tag it as example-image:latest
docker build -t example-serve:latest .


Step 1/11 : FROM ubuntu:18.04
 ---> c090eaba6b94
Step 2/11 : MAINTAINER Amazon AI <sage-learner@amazon.com>
 ---> Using cache
 ---> dd1348b3a447
Step 3/11 : RUN apt-get -y update && apt-get install -y --no-install-recommends          wget          python3-pip          python3-setuptools          nginx          ca-certificates     && rm -rf /var/lib/apt/lists/*
 ---> Using cache
 ---> 5d88b1be1abc
Step 4/11 : RUN ln -s /usr/bin/python3 /usr/bin/python
 ---> Using cache
 ---> 6417ac44ded6
Step 5/11 : RUN ln -s /usr/bin/pip3 /usr/bin/pip
 ---> Using cache
 ---> 09ada8830cc5
Step 6/11 : RUN pip --no-cache-dir install numpy==1.16.2 scipy==1.2.1 scikit-learn==0.20.2 pandas flask gunicorn
 ---> Using cache
 ---> 2885b6a01bd3
Step 7/11 : ENV PYTHONUNBUFFERED=TRUE
 ---> Using cache
 ---> ab2910812f0d
Step 8/11 : ENV PYTHONDONTWRITEBYTECODE=TRUE
 ---> Using cache
 ---> 0c0e511ac354
Step 9/11 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 10fcf3254c9d
Step 10/11 : COPY src /opt/progra

## Test your image

Like in the [notebook for CreateTrainingJob](https://github.com/hsl89/amazon-sagemaker-examples/blob/sagemaker-fundamentals/sagemaker-fundamentals/create-training-job/create-training-job.ipynb), we replicate the Amazon SageMaker hosting environment and test your image locally before serving in production. You are encouraged to read through the section on [Use Your Own Inference Code with Hosting Services](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-inference-code.html) and think about how would you replicate SageMaker hosting environment before moving on. 

Like for `CreateTrainingJob`, SageMaker reserves `/opt/ml` directory in your image to inject ML-related info for `CreateEndpoint`. In particular, it downloads your trained model artifact and inject it in the directory `/opt/ml/model`. When calling `CreateEndpoint` you will need to tell SageMaker the S3 URI of your model artifact. SageMaker will use then pull the artifact and inject it into `/opt/ml/model`. This means when defining your own inference logic, you should load your trained model from `/opt/ml/model`. 

We will use docker python client to run your image and we will mount `container/local_test/ml` to `/opt/ml` as docker volume. 

In [6]:
# look at what's inside `container/ml`
!ls container/local_test/ml

model


The inference logic we implemented in `container/src/predictor.py` under `def inference():` does not require a real ML model. Therefore we do not need to inject anything for the purpose of local test. We will discuss how to load a real model in a more advanced notebook. 

<span style="color:red"> TODO for Dev:  add link to the advanced notebook when it is ready</span>.

#### Run the container

To run the container `example-serve`, open a terminal in the current directory and go to `container/local_test`

```sh
cd container/local_test
```

Then run the following command

```sh
docker run -v ml:/opt/ml -p 8080:8080 --rm example-serve:latest serve 
```

`-v ml:/opt/ml` binds the directory `ml` (in `container/local_test`) to `/opt/ml` in the image as a docker volume.

`-p 8080:8080` exposes port 8080 inside container as port 8080 on the hos

`--rm` removes the container from daemon when it is stopped. 

We suggest you to run the image from the shell instead of within the notebook because when you are debugging your own container, you can more easily stdout from the container when you have a shell process running it. 

#### Ping your container
Once your container is up, you can ping it at `http://localhost:8080`. 

To trigger the logic under `def ping():` in `container/src/predictor.py`, run the following cell

In [65]:
%%sh
curl localhost:8080/ping

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100     1  100     1    0     0   1000      0 --:--:-- --:--:-- --:--:--  1000


To trigger the logic under `def inference():` in `container/src/predictor.py` with a json string, run the following cell

In [64]:
%%sh
curl --header "Content-Type: application/json" \
  --request POST \
  --data '{"key":"value"}' \
  http://localhost:8080/invocations

I am fed with json. Therefore, I am happy

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    56  100    41  100    15  13666   5000 --:--:-- --:--:-- --:--:-- 18666


To stop the container 